In [85]:
#Importing all needed libraries
import pandas as pd
import numpy as np
import sklearn
from scipy.optimize import minimize
import math 

In [ ]:
#Converting sheet with all country data to dataframe
time_series_countries = pd.read_excel('time-series-countries.xlsx').iloc[:, [0,1,2]]
print(time_series_countries)

In [ ]:
#Generate list of countries
country_list = time_series_countries['location'].to_list()
country_list = list(set(country_list)) 
print(country_list)

In [ ]:
#Select country name using user input
country = input("Enter name of country: ")
print(country)
if country not in country_list:
    print("Invalid Input")
    exit()
P0 = input("Enter the population of the chosen country: ")
initinve = input("Enter the initial value of 1/e period of of the chosen country: ")


In [53]:
#Generate new dataframe with country-specific data
country_data = time_series_countries.loc[time_series_countries['location'] == country]
C = country_data['total_cases'].to_list()
date_list = country_data['date'].to_list()
#print(country_data)
#print(date_list)

In [ ]:
#Generate data lists for country
def avgdata(a):
    avga = [a[0]]
    for i in range(1,7):
        avga.append((avga[i - 1]*i + a[i])/(i + 1))
    temp = avga[6]*7
    for i in range(7,len(a)):
        temp = temp + a[i] - a[i - 7]
        avga.append(temp/7)
    return avga
y = 0.1
Rt = [0]
for i in range(1,len(C)):
    Rt.append(round((1 - y)*Rt[i - 1] + y*C[i - 1] + 0.01))
#print(C)
#print(Rt)
T = []
for i in range(len(C)):
    T.append(C[i] - Rt[i])
#print(T)
AvC = avgdata(C)
AvRt = avgdata(Rt)
AvT = avgdata(T)
#print(AvT)
#print(AvRt)
DelC = []
for i in range(7):
    DelC.append(AvC[i])
for i in range(7,len(C)):
    DelC.append(AvC[i] - AvC[i - 7])
IntT = [AvT[0]]
for i in range(1,7):
    IntT.append(IntT[i - 1] + AvT[i])
for i in range(7,len(AvT)):
    IntT.append(IntT[i - 1] + AvT[i] - AvT[i - 7])
#print(IntT)
IntTC = [AvT[0]*AvC[0]]
for i in range(1,7):
    IntTC.append(IntTC[i - 1] + AvT[i]*AvC[i])
for i in range(7,len(AvT)):
    IntTC.append(IntTC[i - 1] +  AvT[i]*AvC[i] - AvT[i - 7]*AvC[i-7])
#print(IntTC)
#print(len(C),len(Rt),len(T),len(AvRt),len(AvC),len(DelC),len(IntT),len(IntTC))


In [51]:
#Take phase inputs
startd = input("Enter the start date of the phase (YYYY-MM-DD format): ")
endd = input("Enter the end date of the phase (YYYY-MM-DD format): ")
drift = input("Enter the drift period of the phase: ")
skip = input("Enter the skip period of the phase: ")
skip = int(skip)
drift = int(drift)
if startd in date_list:
    startd = date_list.index(startd)
else:
    print("Wrong start date input")
if endd in date_list:
    endd = date_list.index(endd)
else:
    print("Wrong end date input")

In [99]:
#Regression
skip = int(skip)
drift = int(drift)
u = np.array(IntT[startd + drift + skip: endd])
v = np.array(DelC[startd + drift + skip + 1 : endd + 1])
w = np.array(IntTC[startd + drift + skip: endd])
w = w/P0
#print(u,v,w)

#type "S"
vw_df = pd.DataFrame({'v':v,'w':w})
LR = sklearn.linear_model.LinearRegression(fit_intercept=False)
LR.fit(vw_df,u)
invbtS = LR.coef_[0]
invptS = LR.coef_[1]
r2_S = LR.score(vw_df,u)
#print(invbtS,invptS,r2_S)
delt = np.dot(v,v)*np.dot(w,w) - np.dot(v,w)*np.dot(v,w)
delt = abs(delt)
var = (np.dot(u,u) - invbtS*np.dot(u,v) - invptS*np.dot(u,w))/(endd - startd - drift - skip - 2)
var = abs(var)
stdbtS = math.sqrt(np.dot(w,w)*var/delt)
stdptS = math.sqrt(np.dot(v,v)*var/delt)
errS = np.dot(u - invbtS*v - invptS*w,u - invbtS*v - invptS*w)/np.dot(u,u)
#print(errS)
toterrS = stdbtS/invbtS + stdptS/invptS + 10*errS
#print(toterrS)

#type "A"
def funA(args):
    x = args[0]
    y = args[1]
    return (np.linalg.norm(u - x*v - y*w)*np.linalg.norm(u - x*v - y*w))/(np.linalg.norm(u - x*v)*np.linalg.norm(u- y*w))
regA = minimize(funA,[1,1])
if regA.success == 0: 
    invbt, invpt = invbtS, invptS
else:
    x, y = regA.x
    varbt = np.dot(u,u) + np.dot(w,w)/(y*y) - 2*np.dot(u,w)/y - pow(np.dot(u,v) - np.dot(v,w)/y,2)/np.dot(v,v)
    varbt = varbt/endd - startd - drift - skip - 1
    varpt = np.dot(u,u) + np.dot(v,v)/(x*x) - 2*np.dot(u,v)/x - pow(np.dot(u,w) - np.dot(v,w)/x,2)/np.dot(w,w)
    varpt = varpt/endd - startd - drift - skip - 1
    stdbtA = math.sqrt(varbt/np.dot(v,v))
    stdptA = math.sqrt(varpt/np.dot(w,w))
    errbt = np.dot(u - x*v - y*w,u - x*v - y*w)/np.dot(u - y*w,u - y*w)
    errpt = np.dot(u - x*v - y*w,u - x*v - y*w)/np.dot(u - x*v,u - x*v)
    #calculate error in A
    toterrA = stdbtA/x + stdptA/y + errbt + errpt
    #print(x,y,toterrA)
    if (invbtS > 10 or invbtS < 1 or invptS < 0 or toterrA < toterrS):
        invbt, invpt = x,y
    else:
        invbt, invpt = invbtS, invptS

#Final result
print(invbt,invpt)

3.151273110695222 486.05566106304974


In [ ]:
#Predicted trajectory